## GND-Provenienz-Suche
### Überblick
Die Suche wird via lobid.org auf die GND durchgeführt. Aus den GND-Datensätzen werden die URLs für die Bildquellen geparst und dann die zugehörigen Bilder aus derzeit zwei Quellen geharvestet: dem GBV-Provenienz-Wiki (https://provenienz.gbv.de/) und der Deutschen Fotothek (https://www.deutschefotothek.de/). Um den Prozess etwas zu beschleunigen, werden alle Bilder verkleinert im Unterordner "static" gecacht und bei neuen Suchen daraus geladen. Die Suche nach 1000 Provenienzmerkmalen dauert bei der ersten Abfrage der Bilder ca. 25 Minuten.
Derzeit gibt es zwei Abfragemodi: Eine reguläre Suche mit Begriffen sowie eine direkte Abfrage von GND-IDs, die als kommagetrennte Liste in den Suchschlitz kopiert werden können.
### Bedienhinweise
Der Code dieser Anwendung ist ausgeblendet und hinter `...` versteckt. Sichtbar sind nur die interaktiven Elemente und die Ergebnisse. Sollten unterhalb dieses Textes keine Bedienelemente zu sehen sein, so aktualisieren Sie die Seite im Menü über `Run => Run all Cells`. Sollten Sie aus Versehen den Code sichtbar machen, so können Sie ihn im Menü über `View => Collapse All Code` wieder ausblenden.
### Benutzung
1. `Suchbegriff`, `Suchmodus` und `Maximale Treffer` wählen. Über lobid.org kann mit `\~` auch unscharf gesucht werden, z. B. `leipzik\~ stenpel\~`. Beim Suchtyp `ID-Liste` kann eine Liste von GND-IDs in den Suchschlitz kopiert werden in der Form `117464012X,1096198665,1273284453,1266422757`. 
2. Auf das Ergebnis klicken, sodass der blaue Balken dort am linken Rand angezeigt wird (Zelle gewählt). Nun `Umschalt+Enter` drücken.
3. Warten. Je nach Umfang der Trefferliste und Stand des Cachings der Bilder kann es einige Sekunden bis viele Minuten dauern, bis die Ergebnisse angezeigt werden.


In [7]:
import requests
import json
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
from bs4 import BeautifulSoup
from io import BytesIO
import re
import urllib.parse
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from IPython.display import display, HTML, Javascript
import ipywidgets as widgets

# generate folder static if missing    
path = 'static'
isExist = os.path.exists(path)
if (isExist == False):
    os.mkdir('static')

# generate arrays if listfiles are missing 
try:
    with open('filelist.json', 'r', errors='ignore') as f:
        filelist = json.load(f)
except FileNotFoundError:
    filelist = []

try:
    with open('errorlist.json', 'r', errors='ignore') as f:
        errorlist = json.load(f)
except FileNotFoundError:
    errorlist = [""]
    
# generate input area
    
searchterm = widgets.Text(
    value='Jena Stempel Universität',
    placeholder='Type something',
    description='Suchbegriffe/IDs:',
    disabled=False
)
display(searchterm)

search_type = widgets.RadioButtons(
    options=['Suche', 'ID-Liste'],
    description='Suchmodus:',
    disabled=False
)
display(search_type)

response_size = widgets.Dropdown(
    options=[1, 10, 50, 250, 1000],
    value=1,
    description='Maximale Treffer:',
    disabled=False,
)
display(response_size)

Text(value='Jena Stempel Universität', description='Suchbegriffe/IDs:', placeholder='Type something')

RadioButtons(description='Suchmodus:', options=('Suche', 'ID-Liste'), value='Suche')

Dropdown(description='Maximale Treffer:', options=(1, 10, 50, 250, 1000), value=1)

In [2]:
#search_type = "Suche"
#searchterm = "leipzig stempel"
#response_size = "10"

gnd_ids = []
gnd_labels = []

if search_type.value == "Suche":
    url = "https://lobid.org/gnd/search?q=" + searchterm.value + "&filter=type:ProvenanceCharacteristic&format=json&size=" + str(response_size.value)
    response = requests.get(url)
    json_data = response.json()
    for item in json_data['member']:
        gnd_ids.append(item['gndIdentifier'])
        gnd_labels.append(item['preferredName'])

if search_type.value == "ID-Liste":
    gnd_ids = searchterm.value.replace(' ','').split(',')
    lobidsearch = ', '.join(['"{0}"'.format(x) for x in gnd_ids])
    url = 'https://lobid.org/gnd/reconcile/'
    data = {'extend':'{"ids":[' + lobidsearch + '],"properties":[{"id":"preferredName"}]}'}
    response = requests.post(url, data=data)
    response = response.json()
    str_list = [entry['str'] for row in response['rows'].values() for entry in row['preferredName']]
    gnd_labels = str_list        

df = pd.DataFrame({'gnd_id': gnd_ids, 'gnd_label': gnd_labels})

def check_url(item):
    gbv_found = False
    try:
        #url = 'http://d-nb.info/gnd/' + item + '/about/marcxml'
        url = 'https://sru.bsz-bw.de/ognd!rec=2?version=1.1&query=pica.gnd+%3D+' + item + '&recordSchema=marcxmlk10os&version=1.1&maximumRecords=1'
        response = requests.get(url)

        xml_string = response.content
        root = ET.fromstring(xml_string)

        for datafield in root.findall(".//{http://www.loc.gov/MARC21/slim}datafield[@tag='670']"):
            subfield = datafield.find(".//{http://www.loc.gov/MARC21/slim}subfield[@code='u']")
            if subfield is not None:
                # get the text content of the subfield element
                link = subfield.text
                # check if the link contains the target string
                if "provenienz.gbv.de" in link:
                    link = re.sub('images\/.*\/', 'Datei:', link)
                    gbv_found = True
                    return(link)
        if gbv_found == False:
            for datafield in root.findall(".//{http://www.loc.gov/MARC21/slim}datafield[@tag='670']"):
                subfield = datafield.find(".//{http://www.loc.gov/MARC21/slim}subfield[@code='u']")
                if subfield is not None:
                    # get the text content of the subfield element
                    link = subfield.text
                    # check if the link contains the target string
                    if "www.deutschefotothek.de" in link:
                        return(link)        
    except Exception as e:
        return None

df['gbv'] = df['gnd_id'].apply(check_url)

def download_and_resize_image(link):
    if link != None:   
        if "provenienz.gbv.de" in link:
            filepath = "static/" + link.replace('https://provenienz.gbv.de/Datei:','').replace('http://provenienz.gbv.de/Datei:','')
            filepath = urllib.parse.unquote(filepath)
            if filepath not in filelist:
                image_url = link.replace('https://provenienz.gbv.de/','https://provenienz.gbv.de/Special:FilePath/').replace('http://provenienz.gbv.de/','https://provenienz.gbv.de/Special:FilePath/')
                response = requests.get(image_url)
                try:
                    image = Image.open(BytesIO(response.content))
                    width, height = image.size
                    if width > height:
                        new_width = 300
                        new_height = int(height * (new_width / width))
                    else:
                        new_height = 300
                        new_width = int(width * (new_height / height))
                    resized_image = image.resize((new_width, new_height))
                    resized_image.save(filepath)
                    filelist.append(filepath)
                except:
                    filepath = "none"
            return filepath

        if "deutschefotothek.de" in link:
            try:
                response = requests.get(link)
                html_content = response.content
                soup = BeautifulSoup(html_content, 'html.parser')
                div_tag = soup.find('div', class_='listItemThumbnail')
                img_tag = div_tag.find('img')
                download_link = img_tag['src']
                #download_link = soup.find('a', {'class': 'download'})
                #download_link = download_link['href']
                filepath = "static/" + download_link.split("/")[-1]
                if filepath not in filelist:
                    image_url = download_link
                    response = requests.get(image_url)
                    try:
                        image = Image.open(BytesIO(response.content))

                        width, height = image.size
                        if width > height:
                            new_width = 300
                            new_height = int(height * (new_width / width))
                        else:
                            new_height = 300
                            new_width = int(width * (new_height / height))
                        resized_image = image.resize((new_width, new_height))
                        resized_image.save(filepath)
                        filelist.append(filepath)
                    except:
                        filepath = "none"
                return filepath

            except Exception as e:
                return None

df['thumbnails'] = df['gbv'].apply(download_and_resize_image)

df['thumb'] = df['gbv'].replace('https\:\/\/provenienz\.gbv\.de\/','https://provenienz.gbv.de/Special:FilePath/', regex=True).replace('http\:\/\/provenienz\.gbv\.de\/','https://provenienz.gbv.de/Special:FilePath/', regex=True)

def format_link(value):
    url = f"https://d-nb.info/gnd/{value}"
    return f'<a href="{url}" target="_blank">{value}</a>'

df['gnd_id'] = df['gnd_id'].apply(format_link)

def format_link(value):
    return f'<a href="{value}" target="_blank">URI des Merkmals</a>'

df['gbv'] = df['gbv'].apply(format_link)

def bold_formatter(value):
    return f'<b>{value}</b>'

if not df.empty:
    df = df.rename(columns={'gbv': 'image_source'})
    df['thumbnails'].fillna("error.svg",inplace=True)
    df['thumbnails'] = df['thumbnails'].apply(lambda x: x.replace("Provenienzsuche/","")).apply(lambda x: x.replace("none","error.svg"))
    df['thumbnails'] = df['thumbnails'].apply(lambda x: f'<img src="{x}" width="250"/>')
    df['gnd_label'] = df['gnd_label'].str.wrap(50) 
    html = df[['gnd_id', 'gnd_label', 'image_source', 'thumbnails']].to_html(formatters={'gnd_label': bold_formatter}, render_links=True, escape=False).replace("\\n", "<br>")
    display(HTML(html))
else:
    print("No results found for search term:", searchterm.value)

df.apply(lambda x: errorlist.append(x["gnd_id"] + " | " + x["image_source"]) if "error.svg" in x["thumbnails"] else None, axis=1)

errorlist = list(set(errorlist))

with open('filelist.json', 'w') as f:
    json.dump(filelist, f)

with open('errorlist.json', 'w') as f:
    json.dump(errorlist, f)


,gnd_id,gnd_label,image_source,thumbnails
0,1273284453,"Stempel (Universitätsbibliothek Jena), 04",URI des Merkmals,
